In [ ]:
import heapq
import numpy as np
from scipy.stats import poisson, expon, pareto, erlang
from collections import namedtuple
from scipy.stats import norm
import math

In [5]:
Event = namedtuple('Event', ['event_type', 'time'])

class EventList:
    def __init__(self):
        self._queue = []

    def enqueue(self, event: Event):
        heapq.heappush(self._queue, (event.time, event))

    def dequeue(self):
        return heapq.heappop(self._queue)[1]

    def __len__(self):
        return len(self._queue)


class HyperExponential:
    def __init__(self, ps, lambdas):
        self.ps = ps
        self.lambdas = lambdas

    def rvs(self):
        i = np.random.choice(len(self.ps), p=self.ps)
        return np.random.exponential(1.0 / self.lambdas[i])
    
class Weibull:
    def __init__(self, k):
        self.k = k

    def rvs(self):
        return np.random.weibull(self.k,1)[0]
    
class ConstantService:
    def __init__(self, value):
        self.value = value

    def rvs(self):
        return self.value
        

class Simulation:
    def __init__(self, arrive_dist, service_dist, n_units=10, limit=10000):
        self.arrive_dist = arrive_dist
        self.service_dist = service_dist
        self.event_list = EventList()
        self.available_units = n_units
        self.n_blocked = 0
        self.n_served = 0
        self.limit = limit

def create_event(event_type: str, world: Simulation, time_offset: float):
    if event_type == 'arrive':
        if hasattr(world.arrive_dist, 'rvs'):
            target_time = world.arrive_dist.rvs()
        else:
            target_time = world.arrive_dist()
    elif event_type == 'service':
        if hasattr(world.service_dist, 'rvs'):
            target_time = world.service_dist.rvs()
        else:
            target_time = world.service_dist()
    else:
        raise ValueError("Invalid event type")
    return Event(event_type, time_offset + target_time)

def handle_arrive(world: Simulation, event: Event):
    if world.n_blocked + world.n_served >= world.limit:
        return

    new_event = create_event('arrive', world, event.time)
    world.event_list.enqueue(new_event)

    if world.available_units > 0:
        world.available_units -= 1
        service_event = create_event('service', world, event.time)
        world.event_list.enqueue(service_event)
    else:
        world.n_blocked += 1

def handle_service(world: Simulation, event: Event):
    world.available_units += 1
    world.n_served += 1

def simulate(world: Simulation):
    initial_event = Event('arrive', 0.0)
    world.event_list.enqueue(initial_event)

    while len(world.event_list) > 0:
        event = world.event_list.dequeue()
        if event.event_type == 'arrive':
            handle_arrive(world, event)
        elif event.event_type == 'service':
            handle_service(world, event)
        else:
            raise ValueError("Unknown event type")
    return world

def confidence_interval(data, alpha=0.05):
    n = len(data)
    mean = np.mean(data)
    std = np.std(data, ddof=1)
    z = norm.ppf(1 - alpha / 2)
    margin = z * std / np.sqrt(n)
    return mean - margin, mean, mean + margin

def runsim(arrive_dist, service_dist, limit=10000, runs=10, units=10):
    results = []
    for _ in range(runs):
        world = Simulation(arrive_dist, service_dist, units, limit)
        result = simulate(world)
        results.append(result)

    blocked_fractions = [w.n_blocked / (w.n_blocked + w.n_served) for w in results]
    ci = confidence_interval(blocked_fractions)

    print(f"Blocked fraction: {ci[1]:.4f}")
    print(f"95% CI: [{ci[0]:.4f}, {ci[2]:.4f}]")
    return ci


## 1 Poisson process blocking system

In [9]:
runsim(lambda: np.random.exponential(1), expon(scale=8))


Blocked fraction: 0.1236
95% CI: [0.1207, 0.1265]


(np.float64(0.12073436513455688),
 np.float64(0.12360472163786122),
 np.float64(0.12647507814116557))

### Exact solution

In [10]:
m = 10 #service units
mean_arrival = 1
mean_service = 8
total_customers = 10_000
A = mean_arrival * mean_service

In [12]:
s = 0
for i in range(m+1):
    s += (A**i) / math.factorial(i)
(A**m/math.factorial(m))/s

0.12166106425295149

## 2 Renewal process 

### Erlang arrival times

In [7]:
for k in [0.1,0.5,1,2]:
    print(f"k = {k}")
    runsim(erlang(1, scale=k), expon(scale=8))

k = 0.1
Blocked fraction: 0.8764
95% CI: [0.8748, 0.8780]
k = 0.5
Blocked fraction: 0.4437
95% CI: [0.4403, 0.4472]
k = 1
Blocked fraction: 0.1210
95% CI: [0.1164, 0.1256]
k = 2
Blocked fraction: 0.0045
95% CI: [0.0039, 0.0050]


### Hyper exponential arrival times

In [17]:
runsim(HyperExponential([0.8, 0.2], [0.8333, 5.0]), expon(scale=8))

Blocked fraction: 0.1369
95% CI: [0.1328, 0.1409]


(np.float64(0.1327999117384095),
 np.float64(0.13687174530951388),
 np.float64(0.14094357888061826))

## 3 Poisson process

### Constant service time

In [8]:
runsim(lambda: np.random.exponential(1), ConstantService(8))

Blocked fraction: 0.1243
95% CI: [0.1223, 0.1262]


(np.float64(0.12232589867281334),
 np.float64(0.12426299586098111),
 np.float64(0.12620009304914886))

### Pareto service times

In [20]:
for k in [1.05,2.05]:
    print(f"k = {k}")
    runsim(lambda: np.random.exponential(1), pareto(b=k, scale=8*(k-1)))


k = 1.05
Blocked fraction: 0.0019
95% CI: [0.0002, 0.0036]
k = 2.05
Blocked fraction: 0.4507
95% CI: [0.4459, 0.4556]


### Weibull service times

In [43]:
for k in [0.1, 0.2, 0.3]:
    print(f"k = {k}")
    runsim(lambda: np.random.exponential(1), Weibull(k))

k = 0.1
Blocked fraction: 0.9897
95% CI: [0.9877, 0.9917]
k = 0.2
Blocked fraction: 0.8650
95% CI: [0.8496, 0.8805]
k = 0.3
Blocked fraction: 0.1692
95% CI: [0.1556, 0.1828]
